## Basic Search Demo

Demonstrates the basic use of the Best-First Search idea on the 8-puzzle.

In [1]:
# imports and setup
import heapq
import math
import numpy as np
import random

from enum import Enum
from IPython.display import Image

### Defining some basic elements of the 8-puzzle domain
We begin by specifying some of the basic elements of the 8-puzzle, including the 4 actions available, and the solution goal-state. States will be encoded as 2-dimensional integer arrays, where 0 is used to represent the blank square.

In [2]:
# Grid constant
GRID_SIZE = 3

# Action constants
class Action(Enum):
    UP = 1
    RIGHT = 2
    LEFT = 3
    DOWN = 4
    
# Goal-state constant
GOAL_STATE = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

In [3]:
def action_is_legal(puzzle_state, action):
    '''Determines whether a given action is legal for a puzzle-state.
       Actions are legal if and only if taking that action would not
       require moving the blank square outside the grid.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       action: some one of the actions available in the enumeration Action
       
       Returns
       -------
       boolean value: true if and only if action is legal in puzzle_state
    '''
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    return (action == Action.UP and blank_row > 0) \
        or (action == Action.DOWN and blank_row < GRID_SIZE - 1) \
        or (action == Action.LEFT and blank_col > 0) \
        or (action == Action.RIGHT and blank_col < GRID_SIZE - 1)

In [4]:
def get_next_state(puzzle_state, action):
    '''Returns next state reached from a state, after an action.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       action: some one of the actions available in the enumeration Action
       
       Returns
       -------
       boolean value: the next state after taking action in puzzle_state;
           represented as 2-dimensional array, like puzzle_state
           
       Exceptions
       ----------
       Exception raised if not action_is_legal(puzzle_state, action)
    '''
    next_state = np.ndarray.copy(puzzle_state)
    
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    if not action_is_legal(puzzle_state, action):
        raise Exception(
            'Action {} not legal; position of blank is [{}, {}]'.format(action.name, blank_row, blank_col))
    
    target_row = blank_row
    target_col = blank_col
    
    if action == Action.UP:
        target_row = target_row - 1
    elif action == Action.DOWN:
        target_row = target_row + 1
    elif action == Action.LEFT:
        target_col = target_col - 1
    elif action == Action.RIGHT:
        target_col = target_col + 1
                
    next_state[blank_row][blank_col], next_state[target_row][target_col] = \
        next_state[target_row][target_col], next_state[blank_row][blank_col]
    
    return next_state

In [5]:
def get_possible_actions(puzzle_state):
    '''Generates lists of possible actions and the states that result from them,
       starting in puzzle_state.
       
       Args
       ----
       puzzle_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       
       Returns
       -------
       action_list: list of Action items legal in puzzle_state
       next_state_list: list of puzzle configurations reachable by taking actions
           Note: each element of next_state_list is the 2-dimensional array
                 configuration that is reached after taking the matching action
                 from action_list in puzzle_state
    '''
    action_list = list()
    next_state_list = list()
    
    is_blank = np.argwhere(puzzle_state==0)
    blank_row = is_blank[0][0]
    blank_col = is_blank[0][1]
    
    for action in Action:
        if action_is_legal(puzzle_state, action):
            action_list.append(action)
            next_state_list.append(get_next_state(puzzle_state, action))
    
    return action_list, next_state_list

### Generating a random starting state
In order to initialize an 8-puzzle instance properly, we need to ensure that the instance is actually solvable.  Simply generating some random arrangement of the "tiles" (0..8) will *not* work in general–if we do so, it is possible that there is no actual way to use legal moves to get to the desired solution.

One way of guaranteeing solvable instances is to begin with the desired goal state, and then make random moves from that configuration.  This guarantees that we can, at worst, reverse that sequence of moves to solve the puzzle; of course, it may be that there is some shorter solution, since our random moves may, for example, repeat configurations along the way.

In [6]:
def generate_random_start_state(random_moves=25):
    '''Generates a random start state for the 8-puzzle.
       To ensure a reachable solution, starts with goal state
       and permutes randomly using legal moves.
       
       Args
       ----
       random_moves: number of moves to make from GOAL_STATE
       
       Returns
       -------
       puzzle_state: puzzle configuration reached after taking random_moves legal actions
           at random from GOAL_STATE
    '''  
    puzzle_state = np.ndarray.copy(GOAL_STATE)
    for i in range(random_moves):
        action_list, next_state_list = get_possible_actions(puzzle_state)
        puzzle_state = random.choice(next_state_list)
        
    return puzzle_state

### Encoding a data structure for search-tree nodes

In [7]:
Image(url="search_tree_node.pdf", width=500)

In [8]:
class TreeNode:
    '''Object for storing search-tree node data.
    
       Members: 
       --------
       state: 8-puzzle configuration as 2-dimensional array
       action: Action item that led to this state in search
           Note: will be None if and only if this is starting state
       cost: depth of node in search-tree
       parent: TreeNode that led to this node in search
           Note: will be None if and only if this is starting state
           
        Implicits
        ---------
        String conversion: defined based upon state.
        Comparison: lt (<) operator compares cost.
        
        Methods
        -------
        isGoal: returns true if and only if this node's state is identical to GOAL_STATE
        
        pathCost: returns (cost + 1) if supplied with node that has self as parent
            Note: will return arbitrarily large value if node has a different parent
    '''
    def __init__(self, state, action=None, cost=0, parent=None):
        self.state = state
        self.action = action
        self.cost = cost
        self.parent = parent
        
    def __lt__(self, next_node):
        return self.cost < next_node.cost
    
    def __str__(self):
        return str(self.state)
    
    def isGoal(self):
        return str(self) == str(GOAL_STATE)
    
    def pathCost(self, next_node):
        if next_node.parent == self:
            return self.cost + 1
        
        return math.inf
    
    '''
    In the basic version of the node class, actions all have the same cost (1 per step in search),
    leading search to always default to breadth-first, since the priority queue will be ordered
    by tree depth.  If some actions actually cost more than others, however, we could still order
    by path-cost, without it being equivalent to tree-depth.  In such cases the best-first search
    algorithm is then identical to Dijkstra's algorithm, searching for a shortest path according
    to total action-cost.  

    Here is an example in which going UP costs twice as much as other actions:
    
    def pathCost(self, next_node):
        if next_node.parent == self:
            if next_node.action == Action.UP:
                return self.cost + 2
            else:
                return self.cost + 1
        
        return math.inf
    '''
    

### Implementing basic best-first search

In [9]:
Image(url="bestFirstSearch.pdf", width=750)

In [10]:
def best_first_search(start_state):
    '''Performs best-first search from start_state, trying to reach GOAL_STATE.
       Order of node expansion given by priority order defined in TreeNode (lt).
       
       Args
       ----
       start_state: a 2-dimensional array containing some permutation of]
           the integers in [0,8]
       
       Returns
       -------
       node: TreeNode with state == GOAL_STATE;
           will be None if no solution can be found from start_state
           (this will not happen here, but can happen if start_state is some
            random permutation of [0,8] rather than one generated by function
            generate_random_start_state())
    ''' 
    node = TreeNode(state=start_state) 
    frontier = [node]
    heapq.heapify(frontier)
    reached = {str(node): node}
    
    while frontier:
        node = heapq.heappop(frontier)
        if node.isGoal():
            return node
        
        next_action_list, next_state_list = get_possible_actions(node.state)
        for i in range(len(next_state_list)):
            child = TreeNode(next_state_list[i], next_action_list[i], 0, node)
            child.cost = node.pathCost(child)
            if (str(child) not in reached) or (child.cost < reached[str(child)].cost):
                reached.update({str(child):child})
                heapq.heappush(frontier, child)
    
    return None

In [11]:
def printSolution(solution_node):
    '''Prints out solution path from start-to-finish, after best-first-search.
       Back-tracks from solution to start and prints the states and actions of 
       a solution path, in (start -> solution) order.
       
       Args
       ----
       solution_node: a TreeNode returned by best_first_search()
    ''' 
    solution_path = [solution_node]
    parent = solution_node.parent
    while parent != None:
        solution_path.insert(0, parent)
        parent = parent.parent
        
    print(solution_path[0])
    for i in range(1, len(solution_path)):
        print('Action {}: {}'.format(i, solution_path[i].action))
        print(solution_path[i])

### Running our solution on some randomly generated problem-instances

In [12]:
start_state = generate_random_start_state()
print("Start state:")
print(start_state)

Start state:
[[1 2 3]
 [7 4 5]
 [8 0 6]]


In [13]:
solution = best_first_search(start_state)
print(solution.state)
print("Cost = {}".format(solution.cost))

[[1 2 3]
 [4 5 6]
 [7 8 0]]
Cost = 5


In [14]:
printSolution(solution)

[[1 2 3]
 [7 4 5]
 [8 0 6]]
Action 1: Action.LEFT
[[1 2 3]
 [7 4 5]
 [0 8 6]]
Action 2: Action.UP
[[1 2 3]
 [0 4 5]
 [7 8 6]]
Action 3: Action.RIGHT
[[1 2 3]
 [4 0 5]
 [7 8 6]]
Action 4: Action.RIGHT
[[1 2 3]
 [4 5 0]
 [7 8 6]]
Action 5: Action.DOWN
[[1 2 3]
 [4 5 6]
 [7 8 0]]
